In [1]:
import os, sys, yaml, tempfile, time
sys.path.append("/Users/matt.meng/dev/rnn_regressor")
import tensorflow as tf
import pandas as pd
from data_preprocess import load_training_data_from_gcs, load_yaml_file_from_gcs, create_train_test_by_index
from google_cloud_storage_util import GCS_Bucket
from series_data_generator import SeriesDataGenerator

In [2]:
gcs_path = 'test/MachineLearning'
yaml_file_name = 'target_median_norm_configuration.yaml'
config_dict, local_data_file = load_training_data_from_gcs(gcs_path, yaml_file_name)

In [3]:
print local_data_file

/var/folders/k5/nkzf3kxj2v721gq7p2nytvv48ppbxr/T/tmp2bnZvQ


In [4]:
index_gcs_path = 'test/MachineLearning/index_yaml'
index_file_name = 'NYDN_hybrid_model_fold_2.yaml'
bucket = GCS_Bucket()
index_dict = load_yaml_file_from_gcs(bucket, index_gcs_path, index_file_name)

In [5]:
train, valid_data = create_train_test_by_index(local_data_file, config_dict, index_dict)


total #columns: 187, training data: 8101, validation data: 8101.


In [6]:
print train.shape
train.head()

(8101, 187)


,total_views,views_0min_to_10min,views_10min_to_20min,views_20min_to_30min,views_30min_to_40min,views_40min_to_50min,views_50min_to_60min,views_60min_to_70min,views_70min_to_80min,views_80min_to_90min,...,pageReferrer_SOCIAL_80min_to_90min,pageReferrer_SOCIAL_90min_to_100min,pageReferrer_SOCIAL_100min_to_110min,pageReferrer_SOCIAL_110min_to_120min,minLocalDateInWeek,minLocalTime,createTime,articleInfo_type,articleInfo_authorName,articleInfo_section
articleId,,,,,,,,,,,,,,,,,,,,,
fe14a0c51595d6b2,1.489762,0.000186,0.000372,0.001489,0.017964,0.011821,0.021314,0.022990,0.028388,0.031087,...,0.000000,0.000000,0.000000,0.000000,0.000093,0.027550,0.029226,1.851089,2.078020,1.594471
b3ccb90b8f256bb3,1.032856,0.001210,0.025875,0.028853,0.047282,0.056869,0.065525,0.057800,0.058917,0.052681,...,0.000000,0.000000,0.000000,0.000093,0.000186,0.066549,0.067573,1.851089,1.899045,2.139606
0b9d5d6103145106,0.865227,0.000186,0.000372,0.009214,0.009028,0.008470,0.012193,0.008656,0.008191,0.015078,...,0.000000,0.000000,0.000000,0.000000,0.000093,0.066549,0.068410,1.851089,1.829996,2.422299
6b82629c96ad1cb1,1.342703,0.001768,0.006701,0.012565,0.009680,0.009773,0.010145,0.008935,0.012379,0.014799,...,0.000372,0.000838,0.000558,0.000745,0.000558,0.025689,0.026061,1.851089,2.840517,1.701381
7d40e971565bd494,0.560499,0.005026,0.008377,0.006329,0.006981,0.008098,0.007539,0.008004,0.010797,0.009028,...,0.000000,0.000000,0.000000,0.000000,0.000558,0.086839,0.087211,1.851089,1.395782,1.701381


In [7]:
#%timeit data_generator = SeriesDataGenerator(train, config_dict)

#### create the `data_generator` and check the content

In [8]:
data_generator = SeriesDataGenerator(train, config_dict)

In [9]:
data_sequence = data_generator.data_sequence

In [15]:
print data_sequence[0].keys()
print '\n \n'
print data_sequence[0]['meta_data']
print '\n \n'
print len(data_sequence[0]['time_series_data'])
print '\n \n'
data_sequence[0]['target']

['meta_data', 'time_series_data', 'target']

 

[9.30752047655e-05, 0.027550260610599998, 0.029225614296400002, 1.8510885138799997, 2.07802012537, 1.5944707547299999]

 

12

 



[1.4897617274799999]

#### check the data content consistency

In [16]:
column_names = data_generator._build_time_series_column_names()

In [17]:
column_names

[['views_0min_to_10min',
  'US_counts_0min_to_10min',
  'sessionReferrer_DIRECT_0min_to_10min',
  'sessionReferrer_SOCIAL_0min_to_10min',
  'sessionReferrer_SEARCH_0min_to_10min',
  'sessionReferrer_OTHER_0min_to_10min',
  'platform_PHON_0min_to_10min',
  'platform_DESK_0min_to_10min',
  'platform_TBLT_0min_to_10min',
  'platform_OTHR_0min_to_10min',
  'pageReferrer_OTHER_0min_to_10min',
  'pageReferrer_SEARCH_0min_to_10min',
  'pageReferrer_DIRECT_0min_to_10min',
  'pageReferrer_EMPTY_DOMAIN_0min_to_10min',
  'pageReferrer_SOCIAL_0min_to_10min'],
 ['views_10min_to_20min',
  'US_counts_10min_to_20min',
  'sessionReferrer_DIRECT_10min_to_20min',
  'sessionReferrer_SOCIAL_10min_to_20min',
  'sessionReferrer_SEARCH_10min_to_20min',
  'sessionReferrer_OTHER_10min_to_20min',
  'platform_PHON_10min_to_20min',
  'platform_DESK_10min_to_20min',
  'platform_TBLT_10min_to_20min',
  'platform_OTHR_10min_to_20min',
  'pageReferrer_OTHER_10min_to_20min',
  'pageReferrer_SEARCH_10min_to_20min',
  'p

In [85]:
data_generator._cur_index = 0
batch_size = 64
offset = 10
step = 8

In [103]:
train_data = data_generator.next_batch(64)

In [104]:
cur_index = data_generator._cur_index

In [105]:
cur_index

320

In [106]:
train.iloc[cur_index - offset:cur_index]['total_views']

articleId
e2170a119f872e73     4.666884
0708b74befedca04     0.378816
88e6b0e4aae7272d     0.624162
04152a5046177496     0.687826
ab2c4aab8edae766     1.643987
5c4112d26ef65009     0.714631
28c3ade76f7e44db    13.314501
c724436de8a77a82     1.346984
f2380f7531c3ffad     2.054449
596727663ec350ad     0.622301
Name: total_views, dtype: float64

In [107]:
train_data.target[-offset:]

[[4.6668838421399998],
 [0.378816083395],
 [0.62416232315700004],
 [0.68782576321700006],
 [1.6439873417700002],
 [0.714631422189],
 [13.314501116900001],
 [1.34698436337],
 [2.05444899479],
 [0.62230081906199997]]

In [115]:
train_data.meta_data[-offset:]

[[9.30752047655e-05,
  0.0692479523455,
  0.0697133283693,
  1.8510885138799997,
  1.89922838822,
  1.6792807602700002],
 [0.000279225614296,
  0.0667349218168,
  0.0679448994788,
  1.8510885138799997,
  1.89922838822,
  1.5944707547299999],
 [0.0,
  0.105361131794,
  0.105640357409,
  1.8510885138799997,
  1.8338940998700002,
  1.7394396100599998],
 [0.0005584512285930001,
  0.0699925539836,
  0.0708302308265,
  1.8510885138799997,
  2.30962367243,
  2.13960613987],
 [0.000279225614296,
  0.123603871929,
  0.123976172748,
  1.8510885138799997,
  1.9798791002,
  2.13960613987],
 [0.000279225614296,
  0.102941176471,
  0.103220402085,
  1.8510885138799997,
  1.89922838822,
  1.87994728572],
 [0.000279225614296,
  0.0271779597915,
  0.0282017870439,
  1.8510885138799997,
  3.35291338501,
  2.13960613987],
 [0.000279225614296,
  0.106664184661,
  0.10712956068499999,
  1.8510885138799997,
  1.89922838822,
  1.2058203028],
 [0.000186150409531,
  0.0779970215934,
  0.0794862248697,
  1.8510

In [112]:
train.iloc[cur_index - offset:cur_index][config_dict["static_columns"]]

,minLocalDateInWeek,minLocalTime,createTime,articleInfo_type,articleInfo_authorName,articleInfo_section
articleId,,,,,,
e2170a119f872e73,0.000093,0.069248,0.069713,1.851089,1.899228,1.679281
0708b74befedca04,0.000279,0.066735,0.067945,1.851089,1.899228,1.594471
88e6b0e4aae7272d,0.000000,0.105361,0.105640,1.851089,1.833894,1.739440
04152a5046177496,0.000558,0.069993,0.070830,1.851089,2.309624,2.139606
ab2c4aab8edae766,0.000279,0.123604,0.123976,1.851089,1.979879,2.139606
5c4112d26ef65009,0.000279,0.102941,0.103220,1.851089,1.899228,1.879947
28c3ade76f7e44db,0.000279,0.027178,0.028202,1.851089,3.352913,2.139606
c724436de8a77a82,0.000279,0.106664,0.107130,1.851089,1.899228,1.205820
f2380f7531c3ffad,0.000186,0.077997,0.079486,1.851089,1.607759,2.139606


In [108]:
train_data.time_series_data[-offset][step]

[0.08302308265080001,
 0.0819992553984,
 0.063104988831,
 0.017218912881599997,
 0.0026991809382,
 0.0,
 0.0400223380491,
 0.0396500372301,
 0.00335070737156,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [109]:
tmp_index = cur_index - offset
print tmp_index
train.iloc[tmp_index][column_names[step]]

310


views_80min_to_90min                        0.083023
US_counts_80min_to_90min                    0.081999
sessionReferrer_DIRECT_80min_to_90min       0.063105
sessionReferrer_SOCIAL_80min_to_90min       0.017219
sessionReferrer_SEARCH_80min_to_90min       0.002699
sessionReferrer_OTHER_80min_to_90min        0.000000
platform_PHON_80min_to_90min                0.040022
platform_DESK_80min_to_90min                0.039650
platform_TBLT_80min_to_90min                0.003351
platform_OTHR_80min_to_90min                0.000000
pageReferrer_OTHER_80min_to_90min           0.000000
pageReferrer_SEARCH_80min_to_90min          0.000000
pageReferrer_DIRECT_80min_to_90min          0.000000
pageReferrer_EMPTY_DOMAIN_80min_to_90min    0.000000
pageReferrer_SOCIAL_80min_to_90min          0.000000
Name: e2170a119f872e73, dtype: float64

In [84]:
train_data.time_series_data[-offset]

[[0.00353685778109,
  0.0034437825763200002,
  0.00167535368578,
  0.0,
  0.0018615040953099998,
  0.0,
  0.0013961280714799998,
  0.0018615040953099998,
  0.000279225614296,
  0.0,
  0.0,
  0.0018615040953099998,
  0.00111690245719,
  0.0005584512285930001,
  0.0],
 [0.0136820551005,
  0.0127513030529,
  0.00967982129561,
  0.00148920327625,
  0.00251303052867,
  0.0,
  0.00400223380491,
  0.00819061801936,
  0.00148920327625,
  0.0,
  0.0,
  0.0024199553239000002,
  0.00893521965748,
  0.000837676842889,
  0.00148920327625],
 [0.0133097542815,
  0.0127513030529,
  0.010052122114700001,
  0.000744601638124,
  0.00251303052867,
  0.0,
  0.00651526433358,
  0.006049888309750001,
  0.000744601638124,
  0.0,
  0.0,
  0.00251303052867,
  0.00912137006701,
  0.0012099776619500001,
  0.00046537602382699993],
 [0.0157297096054,
  0.0149851079672,
  0.011634400595700001,
  0.000372300819062,
  0.0037230081906199997,
  0.0,
  0.0077252419955299995,
  0.00725986597171,
  0.000744601638124,
  0.0

In [43]:
train_data.meta_data[10]

[0.000279225614296,
 0.0819992553984,
 0.08265078183170001,
 1.8510885138799997,
 1.9806503297599998,
 1.87994728572]

In [40]:
train_data.meta_data[10]

[0.000279225614296,
 0.0819992553984,
 0.08265078183170001,
 1.8510885138799997,
 1.9806503297599998,
 1.87994728572]

In [ ]:
train_data.target[0]

In [ ]:
train.iloc[0][column_names[0]]

In [ ]:
%timeit train_data = data_generator.next_batch(256)

In [ ]:
%timeit train_data = data_generator.next_batch(4000)

In [ ]:
data_generator = SeriesDataGenerator(train, config_dict)
test_generator = SeriesDataGenerator(valid_data, config_dict)

In [ ]:
batch_size = data_generator.get_total_counts()

In [ ]:
%timeit data = data_generator.next_batch(batch_size)

In [ ]:
indexes = train.index

In [ ]:
train.loc[indexes[:5]]

In [ ]:
train.head()

In [ ]:
df = train.sample(frac=1)

In [ ]:
df.head()

In [ ]:
df.loc['fe14a0c51595d6b2']

In [ ]:
len(train.index.unique())

In [ ]:
config_dict["static_columns"]

In [ ]:
content = train.loc[indexes[:5]][config_dict["static_columns"]].values.tolist()

In [ ]:
train.loc[indexes[5]][config_dict["label_column"]].tolist()

In [ ]:
content

In [ ]:
[[elem] for elem in train.loc[indexes[:5]][config_dict["label_column"]]]

In [ ]:
len(content)

In [ ]:
meta_data = train.loc[indexes[:5]][config_dict["static_columns"]].values.tolist()
target = [[elem] for elem in train.loc[indexes[:5]][config_dict["label_column"]]]

In [ ]:
meta_data